In [1]:
import numpy as np
import pandas as pd
import math
import scipy.stats
import timeit
from Bio import SeqIO
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
class RGDPTransformer(BaseEstimator,TransformerMixin):
    def __init__(self, op='op13', gap=1):
        self.gap = gap
        self.op = op

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        RAAC_scheme = {
            'op5': [['G'], ['I', 'V', 'F', 'Y', 'W'], ['A', 'L', 'M', 'E', 'Q', 'R', 'K'],
                    ['P'], ['N', 'D', 'H', 'S', 'T', 'C']],
            'op8': [['G'], ['I', 'V'], ['F', 'Y', 'W'], ['A', 'L', 'M'], ['E', 'Q', 'R', 'K'],
                    ['P'], ['N', 'D'], ['H', 'S', 'T', 'C']],
            'op9': [['G'], ['I', 'V'], ['F', 'Y', 'W'], ['A', 'L', 'M'], ['E', 'Q', 'R', 'K'],
                    ['P'], ['N', 'D'], ['H', 'S'], ['T', 'C']],
            'op11': [['G'], ['I', 'V'], ['F', 'Y', 'W'], ['A'], ['L', 'M'], ['E', 'Q', 'R', 'K'], ['P'], ['N', 'D'],
                     ['H', 'S'], ['T'], ['C']],
            'op13': [['G'], ['I', 'V'], ['F', 'Y', 'W'], ['A'], ['L'], ['M'], ['E'], ['Q', 'R', 'K'], ['P'], ['N', 'D'],
                     ['H', 'S'], ['T'], ['C']],
            'op20': ['G', 'I', 'V', 'F', 'Y', 'W', 'A', 'L', 'M', 'E', 'Q', 'R', 'K', 'P', 'N', 'D', 'H', 'S', 'T', 'C']
        }
        groups = RAAC_scheme[self.op]

            
        RGDP_scheme = {
            'op5': ['G', 'I', 'A', 'P', 'N'],
            'op8': ['G', 'I', 'F', 'A', 'E', 'P', 'N', 'H'],
            'op9': ['G', 'I', 'F', 'A', 'E', 'P', 'N', 'H', 'T'],
            'op11': ['G', 'I', 'F', 'A', 'L', 'E', 'P', 'N', 'H', 'T', 'C'],
            'op13': ['G', 'I', 'F', 'A', 'L', 'M', 'E', 'Q', 'P', 'N', 'H', 'T', 'C'],
            'op20': ['G', 'I', 'V', 'F', 'Y', 'W', 'A', 'L', 'M', 'E', 'Q', 'R', 'K', 'P', 'N', 'D', 'H', 'S', 'T', 'C']
        }
        header = RGDP_scheme[self.op]
        reduced_dipeptide = [g1 + g2 for g1 in header for g2 in header]

        result = []

        for seq in X:
            for group in groups:
                for key in group:
                    seq = seq.replace(key, group[0])
                
            myDict = {}
            for t in reduced_dipeptide:
                myDict[t] = 0
            
            total = len(seq) - 1 - self.gap
            for j in range(total):
                myDict[seq[j] + seq[j + 1 + self.gap]] += 1

            RGPC = []
            for key in reduced_dipeptide:
                RGPC.append(myDict[key] / total)
                
            result.append(RGPC)
        
        return np.array(result)

In [3]:
def read_fasta(path, maxlen=1500, encode='token'):
    # read the fasta sequences from input file
    fasta_sequences = SeqIO.parse(open(path), 'fasta')
    sequences = []
    labels = []
    for fasta in fasta_sequences:
        # get name and value of each sequence
        name, sequence = str(fasta.id), str(fasta.seq)
        sequences.append(sequence)
        labels.append(name)

    if encode == 'onehot':
        tk = Tokenizer(num_words=21, char_level=True)
        # Fitting
        tk.fit_on_texts(sequences)
        print(tk.word_index)
        sequences, labels = pad_sequences(tk.texts_to_sequences(sequences), maxlen=maxlen, padding='post',
                                          truncating='post'), np.asarray(labels)
        one_hot_sequences = []
        for sequence in sequences:
            b = np.zeros((maxlen, 21))
            b[np.arange(maxlen), sequence - 1] = 1
            one_hot_sequences.append(b.T)
        return np.asarray(one_hot_sequences), labels
    elif encode == 'token':
        return sequences, labels

    return None

In [4]:
def compute_metrics(y_true, y_prob):
    predicted_labels = np.round(y_prob)
    tn, fp, fn, tp = confusion_matrix(y_true, predicted_labels).ravel()
    acc = (tp + tn) / (tn + tp + fn + fp)
    sen = tp / (tp + fn)
    spe = tn / (tn + fp)
    auc = roc_auc_score(y_true, y_prob)
    mcc = (tp*tn - fp*fn) / math.sqrt((tp+fp) * (tp+fn) * (tn+fp) * (tn+fn))
    return acc, sen, spe, auc, mcc

In [5]:
TFPM_used, _ = read_fasta('data/TFPM_training_dataset_used.txt')
TFPM_unused, _ = read_fasta('data/TFPM_training_dataset_unused.txt')
TFPNM, _ = read_fasta('data/TFPNM_training_dataset.txt')

In [6]:
X_train = TFPNM + TFPM_used + TFPM_unused
y_train = np.append([np.zeros(len(TFPNM), dtype=np.int64)], [np.ones(len(TFPM_used) + len(TFPM_unused), dtype=np.int64)])

In [7]:
TFPM_test, _ = read_fasta('data/TFPM_independent_dataset.txt')
TFPNM_test, _ = read_fasta('data/TFPNM_independent_dataset.txt')   

In [8]:
X_test = TFPNM_test + TFPM_test
y_test = np.append([np.zeros(len(TFPNM_test), dtype=np.int64)], [np.ones(len(TFPM_test), dtype=np.int64)])

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, average_precision_score, roc_auc_score, accuracy_score, matthews_corrcoef
from sklearn.metrics import make_scorer

In [10]:
def custom_score(y_true, y_prob):
    return roc_auc_score(y_true, y_prob)

In [11]:
my_scorer = make_scorer(custom_score, needs_proba=True)

In [12]:
op_list = ['op5', 'op8', 'op9', 'op11', 'op13', 'op20']
gap_list = [0, 1, 2, 3, 4, 5]
gamma_list = [10, 1, 0.1, 0.01, 0.001]
C_list = [0.01, 0.1, 1, 10, 100]

In [13]:
results = []
optimal_auc = 0
optimal_params = []
optimal_me = 0

for op in op_list:
    for gap in gap_list:
        for gamma in gamma_list:
            for C in C_list:
                pipe = Pipeline(steps=[("transformer", RGDPTransformer(op=op, gap=gap)),
                                       ("scaler", StandardScaler()),
                                       ("svm", SVC(gamma=gamma, C=C, probability=True, random_state=0))])
                cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=100)
                scores = cross_val_score(pipe, X_train, y_train, scoring=my_scorer, cv=cv, n_jobs=-1)

                current_auc = np.mean(scores)
                current_me = scipy.stats.sem(scores) * scipy.stats.t.ppf((1 + 0.95) / 2., len(scores)-1)
                results.append([current_auc, current_me, op, gap, gamma, C])
                
                if current_auc > optimal_auc:
                    optimal_auc = current_auc
                    optimal_me = current_me
                    optimal_params = [op, gap, gamma, C]

In [14]:
optimal_pipe = Pipeline(steps=[("transformer", RGDPTransformer(op=optimal_params[0], gap=optimal_params[1])),
                               ("scaler", StandardScaler()),
                               ("svm", SVC(gamma=optimal_params[2], C=optimal_params[3],
                                           probability=True, random_state=0))])

optimal_pipe.fit(X_train, y_train)
y_test_probs = optimal_pipe.predict_proba(X_test)[:,1]
acc, sen, spe, auc, mcc = compute_metrics(y_test, y_test_probs)

print(optimal_auc, optimal_me, acc, sen, spe, auc, mcc, optimal_params)

0.8528502997916599 0.011907355098932851 0.7641509433962265 0.8260869565217391 0.6486486486486487 0.8486094790442618 0.47783388288145434 ['op8', 3, 0.1, 0.01]


In [15]:
R = pd.DataFrame(data=results, columns=["auc", "me", "op", "gap", "gamma", "C"])

In [16]:
R

,auc,me,op,gap,gamma,C
0,0.810840,0.011185,op5,0,10.000,0.01
1,0.812878,0.011204,op5,0,10.000,0.10
2,0.810831,0.011561,op5,0,10.000,1.00
3,0.810081,0.011621,op5,0,10.000,10.00
4,0.810486,0.011538,op5,0,10.000,100.00
...,...,...,...,...,...,...
895,0.765431,0.015431,op20,5,0.001,0.01
896,0.765641,0.015235,op20,5,0.001,0.10
897,0.765335,0.015312,op20,5,0.001,1.00
898,0.759872,0.016215,op20,5,0.001,10.00


In [17]:
pivoted_results = R.pivot_table(values="auc", aggfunc="max", index=["op"], columns=["gap"])

In [18]:
pivoted_results

gap,0,1,2,3,4,5
op,,,,,,
op11,0.844479,0.849165,0.841278,0.849074,0.849089,0.844985
op13,0.839236,0.845302,0.834239,0.848903,0.850220,0.847883
op20,0.841014,0.825529,0.826703,0.835262,0.834211,0.838568
op5,0.813750,0.815906,0.816899,0.830136,0.822511,0.819479
op8,0.832825,0.839790,0.834523,0.852850,0.845248,0.834832
op9,0.837501,0.831869,0.830649,0.843397,0.837629,0.843172
